<span style="color:red; font-family:Helvetica Neue, Helvetica, Arial, sans-serif; font-size:2em;">An Exception was encountered at '<a href="#papermill-error-cell">In [3]</a>'.</span>

In [1]:
from pyspark.sql import *
from pyspark.sql.functions import *
import pandas as pd
import os
import sys

PYSPARK_PYTHON = os.getenv("PYSPARK_PYTHON") 
PYSPARK_DRIVER_PYTHON = os.getenv("PYSPARK_DRIVER_PYTHON")

In [2]:
import pyspark
from delta import *
import json

# Load the configuration JSON file
with open('/usr/local/spark/conf/spark-defaults.json', 'r') as f:
    config = json.load(f)

# Initialize the Spark session builder
builder = pyspark.sql.SparkSession.builder.appName("MyApp1").config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog").config("spark.pyspark.python", PYSPARK_PYTHON)\
    .config("spark.pyspark.driver.python", PYSPARK_DRIVER_PYTHON)

# Read the packages from the text file
packages = []
with open('/usr/local/spark/conf/packages.txt', 'r') as file:
    # Read each line and strip newlines or extra spaces
    packages = [line.strip() for line in file if line.strip()]

# # Add packages to the Spark session configuration
builder.config("spark.jars.packages", ",".join(packages))

# Apply the configurations from the JSON file to the Spark session
for key, value in config.items():
    builder.config(key, value)

<span id="papermill-error-cell" style="color:red; font-family:Helvetica Neue, Helvetica, Arial, sans-serif; font-size:2em;">Execution using papermill encountered an exception here and stopped:</span>

In [3]:
# Spark will automatically use the master specified in spark-defaults.conf
spark = configure_spark_with_delta_pip(builder).getOrCreate()

spark

Py4JJavaError: An error occurred while calling None.org.apache.spark.sql.SparkSession.
: java.lang.IllegalStateException: Cannot call methods on a stopped SparkContext.
This stopped SparkContext was created at:

org.apache.spark.api.java.JavaSparkContext.<init>(JavaSparkContext.scala:58)
java.base/jdk.internal.reflect.NativeConstructorAccessorImpl.newInstance0(Native Method)
java.base/jdk.internal.reflect.NativeConstructorAccessorImpl.newInstance(NativeConstructorAccessorImpl.java:77)
java.base/jdk.internal.reflect.DelegatingConstructorAccessorImpl.newInstance(DelegatingConstructorAccessorImpl.java:45)
java.base/java.lang.reflect.Constructor.newInstanceWithCaller(Constructor.java:500)
java.base/java.lang.reflect.Constructor.newInstance(Constructor.java:481)
py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:247)
py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:374)
py4j.Gateway.invoke(Gateway.java:238)
py4j.commands.ConstructorCommand.invokeConstructor(ConstructorComm

In [ ]:
trgt_path_processed = "/mnt/Category/Category_Parquet/"
trgt_path_csv = "/mnt/Category/Category_Processed/"
source_path = "/data/Project_Files/Template.xlsm"
sheet_name ="SPENDING_HISTORY"


In [ ]:
# Read the Excel file (use Spark-Excel library)
df = pd.read_excel(source_path, sheet_name = sheet_name)
df = spark.createDataFrame(df)
df= df.withColumn("Date", to_date(df["Date"],"yyyy-MM-dd"))

In [ ]:
df.createOrReplaceTempView("vw_src")
query = "select distinct Category from vw_src"
print(query)
df_src = spark.sql(query)
df_src.show()

In [ ]:
df_output = \
df_src.withColumn("categorysk",xxhash64("category"))\
        .withColumn("UpdateTimeStamp", date_format(current_timestamp(), "yyyy-MM-dd HH:mm:ss").cast("timestamp"))\
            .withColumn("RowSK",xxhash64(concat_ws("|", *[col(c) for c in df_src.columns])))

In [ ]:
df_output.createOrReplaceTempView("vw_source")

In [ ]:
column_name = df_output.columns
set_clause = ", ".join([f"target.{i} = source.{i}" for i in column_name])
query = f"MERGE INTO delta.`{trgt_path_processed}` AS target USING vw_source AS source ON target.categorysk = source.categorysk AND target.RowSK <> source.RowSK WHEN MATCHED THEN UPDATE SET {set_clause}"
print(query)

In [ ]:
if DeltaTable.isDeltaTable(spark, trgt_path_processed):
    spark.sql(query)
else :
    spark.sql(f"CREATE TABLE delta.`{trgt_path_processed}` USING DELTA AS SELECT * FROM vw_source")

In [ ]:
# Save the DataFrame to a CSV file
spark.read.format("delta").load(trgt_path_processed).coalesce(1).write.format("csv").option("header","true").mode("overwrite").save(trgt_path_csv)

In [ ]:
import shutil
trgt_copy_path = "/mnt/Category/Category.csv"
files=os.listdir(trgt_path_csv)
selected_files = [file for file in files if file.startswith('part-00') and file.endswith('.csv')]
file=trgt_path_csv + selected_files[0]
print(selected_files)
shutil.copy(file, trgt_copy_path)